# Proyecto de Procesamiento de Lenguaje Natural: Question Answering con Tapex

En este proyecto, el objetivo es familiarizarse con el modelo TAPEX, desarrollado por (Qian et al., 2022) como base de su paper sobre la tarea de question answering utilizando tablas.

A lo largo del cuaderno iré probando distintas versiones del modelo con el conjunto de test del dataset Wikitablequestions.

# Importar librerías

Para el desarrollo del proyecto vamos a necesitar dos librerías que no están instaladas, por lo que en la siguiente celda, instalo dos librerías:



*   Librería transformers de HuggingFace con la que podremos importar los modelos de TAPEX.
*   Librería datasets, también de HuggingFace, que permite cargar datasets de forma sencilla.



In [1]:
!pip install transformers
!pip install datasets
!pip install transformers[torch]
!pip install datasets --upgrade

Las librerías las he ido añadiendo según lo iba necesitando a lo largo del proyecto en la siguiente celda:

In [2]:
import pandas as pd
from transformers import TapexTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, SubsetRandomSampler
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import itertools
from itertools import chain
from tqdm import tqdm
import random
import re

# Obtener el dataset


Para tomar el dataset que vamos a utilizar había dos opciones. Descargar el dataset entero y luego importarlo desde mi Drive, o hacer la carga de datos directamente desde su librería de HuggingFace.

Tanto por la comodidad como por la rapidez con la que se puede conseguir el dataset, me he decantado por la segunda opción.

Con el método ```load_dataset()``` de la librería datasets es posible cargar el conjunto de datos simplemente especificando el nombre que tiene este en HuggingFace.



In [3]:
dataset = load_dataset("wikitablequestions")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for wikitablequestions contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikitablequestions
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Vamos a inspeccionar un poco el conjunto de datos que tenemos antes de ir con el modelo.

Ejecutando la siguiente línea podemos observar que el dataset ya está dividido en tres conjuntos: train, test y validation. Está dividido en un 61% para el conjunto de entrenamiento, un 23.5% para el conjunto de test y un 15.5% para el conjunto de validación.

In [ ]:
dataset.items()

dict_items([('train', Dataset({
    features: ['id', 'question', 'answers', 'table'],
    num_rows: 11321
})), ('test', Dataset({
    features: ['id', 'question', 'answers', 'table'],
    num_rows: 4344
})), ('validation', Dataset({
    features: ['id', 'question', 'answers', 'table'],
    num_rows: 2831
}))])

Vamos a dividir ya el conjunto en sus tres particiones.

In [4]:
training_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

Viendo las características de cualquiera de las tres particiones, podemos ver que cada muestra tiene un id, una pregunta, una respuesta y una tabla (que tiene el encabezado o columnas, las filas y un nombre).

In [ ]:
print(training_data.features)

{'id': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'table': {'header': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'rows': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None), 'name': Value(dtype='string', id=None)}}


# Cómo vamos a pasar las tablas a dataframes

El primero de los problemas que tenemos que resolver es el formato de las tablas, ya que según he leído en la documentación, es necesario pasar la tabla como un dataframe, pero ya hemos visto que no tienen este formato.

Para hacer la prueba, he cogido la tabla de una de las muestras al azar. Sabiendo que los dataframes de la librería pandas necesitan dos argumentos (uno con la información de las filas y otro el de las columnas), simplemente he tomado cada parte de la tabla por separado y he creado el dataframe con ellas.

In [ ]:
table = training_data['table'][90]
header = table['header']
rows = table['rows']

df = pd.DataFrame(rows, columns=header)
print(df)

                  Institution                     Location Athletic nickname  \
0            Brown University    Providence,\nRhode Island             Bears   
1         Columbia University          New York,\nNew York             Lions   
2          Cornell University            Ithaca,\nNew York           Big Red   
3           Dartmouth College      Hanover,\nNew Hampshire         Big Green   
4          Harvard University    Cambridge,\nMassachusetts           Crimson   
5        Princeton University       Princeton,\nNew Jersey            Tigers   
6  University of Pennsylvania  Philadelphia,\nPennsylvania           Quakers   
7             Yale University      New Haven,\nConnecticut          Bulldogs   

  Undergraduate enrollment Graduate enrollment Total enrollment  \
0                    6,316               2,333            8,649   
1                    7,160              15,760           22,920   
2                   13,931               6,702           20,633   
3          

Vemos que ha funcionado.

Entonces, podemos generalizar esta técnica y utilizarla para todas las tablas que nos vayamos a encontrar.

He creado la siguiente función para hacerlo:

In [5]:
def change_table_df(table):
  header = table['header']
  rows = table['rows']

  table = pd.DataFrame(rows, columns=header)

  return table

También vamos a necesitar una función que devuelva toda la información (pregunta, respuesta y tabla) de una muestra cuando le pasemos el id de la que queremos.

In [6]:
def get_query_table(dataframe, id):
  query = dataframe['question'][id]
  table = dataframe['table'][id]
  real_answer = dataframe['answers'][id]

  table = change_table_df(table)

  return (query, table, real_answer)

# Z1: Coger el modelo

Una vez hemos conseguido formatear los datos como vamos a necesitarlos, podemos coger el modelo que vamos a usar.

Para las primeras pruebas, vamos a utilizar un modelo preentrenado del TAPEX, y por ahora vamos a tomar su versión Base.

El modelo lo podemos importar utilizando la clase BartForConditionalGeneration, que simplemente es un modelo BART con modelado de lenguaje. Tenemos que especificar de dónde vamos a coger el modelo, y este está en una repositorio de Microsoft, el cual tiene varias versiones del mismo. Como ya he dicho, quiero utilizar la versión base preentrenada.

Además, hay que utilizar un tokenizador para transformar los datos antes de pasarlos al modelo. Este lo podemos conseguir desde la clase TapexTokenizer. Este tokenizador se basa en el Byte-Pair-Encoding, que es una técnica de comprensón de datos.

El tokenizador llevará a cabo el proceso que explico en el informe del modelo, va a aplanar la tabla y concatenarla con la pregunta en una secuencia de tokens.

In [ ]:
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-base-finetuned-wikisql")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-base-finetuned-wikisql")

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

Necesitamos ajustar el tamaño máximo del tokenizador al máximo de embeddings de posición permitidos por nuestro modelo, ya que no funcionará en algunos casos en los que sobrepase el máximo establecido.

In [ ]:
print('Max Position Embeddings:', model.config.max_position_embeddings)

Max Position Embeddings: 1024


# Función para pasar la pregunta y la tabla

Ahora, teniendo en cuenta lo anterior, podemos crear una función que realice la tarea de question answering. Esta recibe una pregunta y una tabla, se las pasa al codificador, luego al modelo y recibe el resultado decodificado.

In [7]:
def question_answering(query, table):
  encoding = tokenizer(table=table, query=query, return_tensors="pt", truncation=True, padding=True, max_length=model.config.max_position_embeddings)

  outputs = model.generate(**encoding)

  result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

  generated_tokens = [string.split() for string in result]

  return result

Para hacer una primera prueba y comprobar que obtenemos es predicción, voy a buscar la pregunta, tabla y respuesta de una muestra aleatoria especificando su id. Ya tengo creada la función que hará esto de antemano.

In [ ]:
id = 23
query, table, real_answer = get_query_table(training_data, id)

Podemos ver que el formato de la pregunta es un string y la tabla es un dataframe.

In [ ]:
print(type(query))
print(type(table))

<class 'str'>
<class 'pandas.core.frame.DataFrame'>


Vamos a ver cuál es la pregunta y la tabla:

In [ ]:
print(query)
print(table)

how many beta versions were released before the first full release?
   Version              Date   Development cycle Size (in kb)    Download  \
0      0.1       9 June 1993                Beta            ?       evolt   
1      0.2      14 June 1993                Beta            ?           ?   
2      0.3      16 June 1993                Beta            ?           ?   
3      0.4      18 June 1993                Beta            ?           ?   
4      0.5      24 June 1993                Beta            ?           ?   
5      0.6      30 June 1993                Beta            ?           ?   
6      0.8   5 November 1993                Beta          N/A         N/A   
7      0.9  12 November 1993            Beta-pre            ?         [2]   
8      0.9  16 November 1993                Beta            ?         [3]   
9      0.9  22 November 1993  WINSOCK alpha r9.2            ?         [4]   
10     1.0  17 February 1994             Release            ?       evolt   
11    1.

Ahora, al ver la predicción y el label, vemos que ha fallado. Pero visto el formato en el que están, nunca va a acertar, porque los formatos de ambos son distintos.

In [ ]:
print(question_answering(query, table))
print('Real answer: ', real_answer)

[' 6.0']
Real answer:  ['9']


# Z2: Evaluando el modelo

Visto lo anterior, va a hacer falta una función para transformar los resultados predichos y los resultados reales al mismo formato.

Primero he creado una función general que va a tomar todas las predicciones y los labels en dos listas (ya que no vamos a estar haciendo predicciones una por una), y les aplica una función de transformación a cada una de ellas.

Las transformaciones que son necesarias las he hecho en base a lo que he ido viendo a lo largo de probar con distintas partes del conjunto de train.

La función para las predicciones cuenta con las siguientes transformaciones:

*   Cambiar todos los strings a minúsculas, para tener un formato común en ambas.
*   Eliminar el primer espacio en blanco que se creaba en todas.
*   Eliminar ciertos carácteres que aparecían de forma común como \\n y \xa0.
*   Cambiar las predicciones que tenían un número como 6.0 a su formato entero, 6 en este caso. He necesitado una función auxiliar para comprobar si un número tiene la parte decimal 0.

La función para los labels tiene las siguientes transformaciones:

*   Cambiar todos los strings a minúsculas también.
*   Cambiar a enteros los floats con parte decimal 0.



In [8]:
def transformar_respuestas(predicciones, respuestas):
  predicciones_transf = [transformar_pred(pred) for pred in predicciones]
  respuestas_transf = [transformar_res(res) for res in respuestas]

  return predicciones_transf, respuestas_transf

def transformar_pred(prediccion):
  nueva_prediccion = [item.lower().lstrip().replace('\\n', '').replace('\xa0', '') for item in prediccion]

  # Agregar transformación para números con parte decimal 0
  nueva_prediccion = [str(int(float(item))) if es_numero_entero(item) else item for item in nueva_prediccion]

  return nueva_prediccion

def es_numero_entero(numero):
  # Verificar si el formato parece ser un número con parte decimal igual a 0
  patron = re.compile(r'^-?\d+\.0$')
  return bool(patron.match(numero))

def transformar_res(respuesta):
  resp_transf = [item.lower() for item in respuesta]

  resp_transf = [str(int(float(item))) if es_numero_entero(item) else item for item in resp_transf]

  return resp_transf

Vamos a probar las funciones para un caso concreto.

In [ ]:
id = 110
query, table, real_answer = get_query_table(test_data, id)

In [ ]:
print(query)
print(table)

how many hard surface courts are there?
      Outcome  No.               Date  \
0   Runner-up   1.      15 April 2001   
1      Winner   1.       29 July 2001   
2   Runner-up   2.       20 July 2003   
3      Winner   2.         2 May 2004   
4   Runner-up   3.         1 May 2005   
5   Runner-up   4.      30 April 2006   
6      Winner   3.        21 May 2006   
7      Winner   4.       16 July 2006   
8   Runner-up   5.    14 January 2007   
9      Winner   5.      5 August 2007   
10  Runner-up   6.  16 September 2007   
11     Winner   6.     7 October 2007   
12  Runner-up   7.       15 June 2008   
13     Winner   7.       13 July 2008   
14     Winner   8.   14 February 2009   
15     Winner   9.   22 February 2009   
16     Winner  10.    6 February 2011   
17     Winner  11.      14 April 2013   
18     Winner  12.       28 July 2013   

                                     Tournament   Surface           Opponent  \
0     Grand Prix Hassan II, Casablanca, Morocco      Clay  

Vemos que la predicción y la solución son ambas números, pero uno está en float y el otro en int.

In [ ]:
pred = question_answering(query, table)

print(pred)
print('Real answer: ', real_answer)

[' 1.0']
Real answer:  ['3']


Si aplicamos la transformación deberíamos conseguir ambos como enteros.

In [ ]:
print(transformar_pred(pred))
print(transformar_res(real_answer))

['1']
['3']


También ocurre que, en cierto casos muy puntuales, la pregunta tiene alguna errata, por lo que confunde al modelo y este no es capaz de resolverlo bien.

En este ejemplo, podemos ver que al preguntar por cuándo quedó primero, escribe 'fist' en vez de 'first'. Vamos a ver cómo puede afectar esto al modelo.

In [ ]:
id = 5
query, table, real_answer = get_query_table(test_data, id)

print(query)
print(table)

in which competition did hopley finish fist?
   Year                 Competition                               Venue  \
0  2000  World Junior Championships                     Santiago, Chile   
1  2003            All-Africa Games                      Abuja, Nigeria   
2  2003            All-Africa Games                      Abuja, Nigeria   
3  2004       African Championships  Brazzaville, Republic of the Congo   
4  2004               Olympic Games                      Athens, Greece   
5  2006          Commonwealth Games                Melbourne, Australia   
6  2006          Commonwealth Games                Melbourne, Australia   
7  2007            All-Africa Games                    Algiers, Algeria   
8  2008       African Championships               Addis Ababa, Ethiopia   

  Position         Event    Notes  
0      1st  Discus throw  59.51 m  
1      5th      Shot put  17.76 m  
2      2nd  Discus throw  62.86 m  
3      2nd  Discus throw  63.50 m  
4      8th  Discus throw

Aquí lo tenemos, responde mal. Veamos qué ocurre si corrijo a mano la pregunta.

In [ ]:
pred = question_answering(query, table)

print(transformar_pred(pred))
print(transformar_res(real_answer))

['african championships']
['world junior championships']


Ahora responde correctamente. Se puede ver que una pequeña errata en la pregunta afecta mucho al modelo. Pero es imposible repasar todas las muestras una por una para determinar cuáles tienen erratas, por lo que es un margen de error que tendremos que asumir.

In [ ]:
query = 'in which competition did hopley finish first?'

pred = question_answering(query, table)

print(transformar_pred(pred))
print(transformar_res(real_answer))

['world junior championships']
['world junior championships']


# Z2: Primeras pruebas sobre el conjunto de test

Vamos a probar ahora todo sobre un pequeño conjunto de 50 muestras. Estas las cojo de forma aleatoria

A veces recibía un error con los labels que he conseguido solucionar utilizando la librería MultiLabelBinarizer, que transforma una lista de tuplas o sets a un formato más intuitivo para datos multi-label.

El accuracy que obtenemos es de 0.18, bastante bajo comparado con el 0.47 que obtienen los desarrolladores con la versión de finetuning.

Aún así, la muestra que hemos utilizado es muy pequeña.

In [ ]:
sample_size = 50
sample_indices = random.sample(range(len(test_data)), sample_size)
all_predictions = []
true_labels = []

for id in tqdm(sample_indices, desc='Procesando'):
    query, table, real_answer = get_query_table(test_data, id)

    prediction = question_answering(query, table)

    all_predictions.append(prediction)
    true_labels.append(real_answer)

print(all_predictions)
print(true_labels)

all_predictions, true_labels = transformar_respuestas(all_predictions, true_labels)

print('Resultados transformados:')
print(all_predictions)
print(true_labels)

mlb = MultiLabelBinarizer()
true_labels_binary = mlb.fit_transform(true_labels)
all_predictions_binary = mlb.transform(all_predictions)

accuracy = accuracy_score(true_labels_binary, all_predictions_binary)
print(f"Accuracy: {accuracy}")

Procesando: 100%|██████████| 50/50 [03:54<00:00,  4.70s/it]

[[' david russell'], [' 4th'], [' miguel ángel montes'], [' 9.0'], [' 72019.0'], [' ethylene oxide'], [' cataraqui town centre\\nd downtown'], [' thea'], [' 31 october 2008, 31 october 2008, 1 november 2008, 1 november 2008, 2 november 2008, 2 november 2008, 2 november 2008, 13 february 2009, 13 february 2009, 13 february 2009, 13 february 2009, 30 october 2009, 30 october 2009, 1 november 2009, 1 november 2009, 1 november 2009'], [' richard huw'], [' levanger, markabygd, okkenhaug, ytterøy, åsen'], [' 6.0'], [' 25 january 1952'], [' 6.0'], [' joshua humphreys'], [' 1.0'], [' 12.0'], [' tower'], [' eddie hart'], [' 1.0'], [' 1974.0'], [' hana yori dango 2, miracle in the pacific, youkame no semi, youkame no semi, miracle in the pacific, ohisama, youkame no semi, youkame no semi'], [' plymouth'], [' kitahiroshima'], [' 1.0'], [' sam originally lived and worked in cambridge, but moved to london at'], [' 4.0'], [' 1.0'], [' busan, south korea'], [' jhonny arteaga'], [' united states'], ['


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['1', '12', '13', '20', '31 october 2008, 31 october 2008, 1 november 2008, 1 november 2008, 2 november 2008, 2 november 2008, 2 november 2008, 13 february 2009, 13 february 2009, 13 february 2009, 13 february 2009, 30 october 2009, 30 october 2009, 1 november 2009, 1 november 2009, 1 november 2009', '42515', '4th', '72019', 'analysis publications', 'busan, south korea', 'cataraqui town centred downtown', 'eddie hart', 'ethylene oxide', 'hana yori dango 2, miracle in the pacific, youkame no semi, youkame no semi, miracle in the pacific, ohisama, youkame no semi, youkame no semi', 'jhonny arteaga', 'joshua humphreys', 'kitahiroshima', 'levanger, markabygd, okkenhaug, ytterøy, åsen', 'may 20, 2050', 'michael campbell, mark hensby', 'miguel ángel montes', 'none', 'olney, sharp', 'richard huw', 'sam originally lived and worked in cambridge, but moved to london at', 'tamera campbell'

Esto mismo que hemos hecho se puede hacer mejor utilizando un Dataloader que pasará batches de muestras al modelo.

Además, este Dataloader puede recibir directamente el sampler que podemos crear para el conjunto entero de test. Este se encargará de hacer el sampling aleatorio del número de muestra que especifiquemos.

Este dataloader necesita una función en la que especifiquemos cómo se procesa cada uno de los batches. Le diremos que simplemente tome las queries, tables y labels de las muestras que tiene en su batch.

In [9]:
def collate_fn(batch):
  queries = [item['question'] for item in batch]
  tables = [pd.DataFrame(item['table']['rows'], columns=item['table']['header']) for item in batch]
  labels = [item['answers'] for item in batch]

  return queries, tables, labels

Ahora, en el proceso de predicción, podemos especificar el tamaño de cada uno de los batches y cuántos de ellos queremos. Con estos valores podremos modificar el tamaño del muestreo.

He probado con la mitad del conjunto de test que ya han sido 6 horas de ejecución.

El resultado de acierto es muy bajo, un 18.05%.

In [ ]:
batch_size = 36

sample_size = 2200
sample_indices = random.sample(range(len(test_data)), sample_size)
sampler = SubsetRandomSampler(sample_indices)

dataloader = DataLoader(test_data, batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)

all_predictions = []
true_labels = []

for batch in tqdm(dataloader, desc='Procesando'):
  queries, tables, labels = batch
  batch_predictions = question_answering(queries, tables)

  all_predictions.extend(batch_predictions)
  true_labels.extend(labels)

all_predictions = [[item] for item in all_predictions]
print(all_predictions)
print(true_labels)

Procesando: 100%|██████████| 62/62 [6:03:58<00:00, 352.24s/it]

[[' 6.574 million'], [' 1.0'], [' at\xa0#5\xa0michigan'], [' bell (bel)'], [' 1.0'], [' 1.0'], [' 1.0'], [' 12.0'], [' 18.0'], [' 1.0'], [' 3.0'], [' 25.0'], [' none'], [' 1.0'], [' jupiler league, jupiler league'], [" sidney smith, ralph foster, ralph foster, ralph foster, sam costen, sam costen, l. s. letellier, george c. rogers, george c. rogers, harvey o'brien, harvey o'brien, harvey o'brien"], [' 2.0'], [' 1994.0'], [' 159074.0'], [' ―, ―, ―'], [' 10 games† (5 pre-season, 5 regular season)'], [' 23.0'], [' 1.0'], [' james b. ray'], [' 6.0'], [' marco andretti'], [' 0.0'], [' 1973.0'], [' republican'], [' mr a.j.law'], [" john o'flynn"], [' 1.0'], [' your love is electric'], [' 7.0'], [' 1.0'], [' blue,,,,, orange/brown,, green, white, white,, '], [' greece'], [' 1969.0'], [' athens international film festival'], [' vidant bertie hospital'], [' analysis publications'], [' giants stadium'], [' bradford bulls (2014 season)'], [' -'], [' 1.0'], [' total'], [' 43668.0'], [' 69-70-76-69

Una vez tenemos las predicciones, se puede calcular el accuracy.

In [ ]:
print(all_predictions)
print(true_labels)

all_predictions, true_labels = transformar_respuestas(all_predictions, true_labels)

print('Resultados transformados:')
print(all_predictions)
print(true_labels)

mlb = MultiLabelBinarizer()
true_labels_binary = mlb.fit_transform(true_labels)
all_predictions_binary = mlb.transform(all_predictions)

accuracy = accuracy_score(true_labels_binary, all_predictions_binary)
print(f"Accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', '"', '" cultural interchange"', '"24/7"', '"class of beverly hills"', '"forbidden fruit", "i\'d be your fool again", "i\'m in love with my wife", "you touched my heart", "dearly loved", "a world called you", "so much in love with you"', '"forbidden fruit", "i\'d be your fool again", "i\'m in love with my wife", "you touched my heart", "dearly loved", "a world called you", "so much in love with you", "i wake up in heaven", "it\'ll be her"', '"goodbye city...hello country"', '"hog", "where y\'all from", "fuckin wit uh house party", "the shadiest one", "can\'t hold back", "keep hustlin", "just clownin\'", "the autobiography", "worldwide gunnin\'"', '"ironic"', '"it ain\'t a thang"', '"o janewale"', '"once upon a dream", "that\'s how you know", "some day my prince will come", "colors of the wind", "reflection", "so this is love", "kiss the girl", "it\'s not just make believe", 

[[' 6.574 million'], [' 1.0'], [' at\xa0#5\xa0michigan'], [' bell (bel)'], [' 1.0'], [' 1.0'], [' 1.0'], [' 12.0'], [' 18.0'], [' 1.0'], [' 3.0'], [' 25.0'], [' none'], [' 1.0'], [' jupiler league, jupiler league'], [" sidney smith, ralph foster, ralph foster, ralph foster, sam costen, sam costen, l. s. letellier, george c. rogers, george c. rogers, harvey o'brien, harvey o'brien, harvey o'brien"], [' 2.0'], [' 1994.0'], [' 159074.0'], [' ―, ―, ―'], [' 10 games† (5 pre-season, 5 regular season)'], [' 23.0'], [' 1.0'], [' james b. ray'], [' 6.0'], [' marco andretti'], [' 0.0'], [' 1973.0'], [' republican'], [' mr a.j.law'], [" john o'flynn"], [' 1.0'], [' your love is electric'], [' 7.0'], [' 1.0'], [' blue,,,,, orange/brown,, green, white, white,, '], [' greece'], [' 1969.0'], [' athens international film festival'], [' vidant bertie hospital'], [' analysis publications'], [' giants stadium'], [' bradford bulls (2014 season)'], [' -'], [' 1.0'], [' total'], [' 43668.0'], [' 69-70-76-69

# Z3: fine-tuning de un modelo vanilla

Como se especifica en el enunciado de la práctica, el objetivo de la parte 3 es hacer un fine-tuning de un modelo vanilla (sin entrenar)

Cogemos el modelo y tokenizador en su versión vanilla.

In [ ]:
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-base")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-base")

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/988 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Para hacer fine tuning hay un código de python, que activamos con este script.

El script ejecutará dos programas que tenemos que guardar en algún sitio. Yo los he metido en una carpeta que especifico al principio.

Después, para guardar el modelo, hay que especificar un directorio, así luego lo podremos utilizar. Yo he creado una carpeta llamada model.

Además, he quitado todos los comandos que aparecían en el script relacionados con el proceso de evaluación, ya que la haré después como lo hemos hecho con la versión preentrenada.

Podemos especificar cuántos pasos dará en el proceso de entrenamiento. Esto lo cambio en base a cuánto tiempo quiero tener el modelo entrenando, y tendrá relación directa con la calidad de este.

In [ ]:
!export EXP_NAME=model_finetuned

!python /content/drive/MyDrive/Proyecto_PLN/code/run_wikitablequestions_with_tapex.py \
  --do_train \
  --output_dir /content/model/$EXP_NAME \
  --model_name_or_path microsoft/tapex-base \
  --overwrite_output_dir \
  --per_device_train_batch_size 4 \
  --gradient_accumulation_steps 8 \
  --per_device_eval_batch_size 4 \
  --learning_rate 3e-5 \
  --logging_steps 10 \
  --save_steps 500 \
  --warmup_steps 500 \
  --predict_with_generate \
  --num_beams 5 \
  --weight_decay 1e-2 \
  --label_smoothing_factor 0.1 \
  --max_steps 2000

2023-12-14 19:25:48.517124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 19:25:48.517185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 19:25:48.518511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 19:25:49.635087: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/14/2023 19:25:51 - WARNING - __main__ -   Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
12/14/2023 19:25:51 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_g

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Ahora, podemos importar el modelo que hemos finetuneado para evaluarlo.

In [ ]:
model_path = "/content/model/"

tokenizer = TapexTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

He creado una función propia para calcular el acierto.

In [10]:
def calculate_accuracy(true_labels, predictions):
  if len(true_labels) != len(predictions):
        raise ValueError("Las listas deben tener la misma longitud")

  accuracy = 0.0

  for true_label, prediction in zip(true_labels, predictions):
      if prediction == true_label:
          accuracy += 1

  accuracy /= len(true_labels)

  return accuracy

Probamos el modelo finetuned haciendo lo mismo que antes con el modelo preentrenado.

In [ ]:
batch_size = 36

sample_size = 500
sample_indices = random.sample(range(len(test_data)), sample_size)
sampler = SubsetRandomSampler(sample_indices)

dataloader = DataLoader(test_data, batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)

all_predictions = []
true_labels = []

for batch in tqdm(dataloader, desc='Procesando'):
  queries, tables, labels = batch
  batch_predictions = question_answering(queries, tables)

  all_predictions.extend(batch_predictions)
  true_labels.extend(labels)

all_predictions = [[item] for item in all_predictions]
print(all_predictions)
print(true_labels)

Procesando: 100%|██████████| 14/14 [54:52<00:00, 235.17s/it]

[[' 3'], [' 7'], [' 2001'], [' mr b. owen- jones'], [' new zealand'], [' 2013'], [' 100,000'], [' 4'], [' alco'], [' škoda octavia'], [' d'], [' afar'], [' 11'], [' feb 1 2013'], [' 209,945'], [' alamance regional medical center'], [' seattle international film festival'], [' lokomotiv moscow'], [' frank w. smith'], [' 2'], [' amager'], [' 4'], [' 2'], [' surround stakes, warwick stakes'], [' henry j. kaiser'], [' 1p/halley'], [' paul tracy'], [' brabham-repco'], [' 1 year'], [' 88'], [' você decide'], [' ragamuffin'], [' bob lemon'], [' 1'], [' niall english'], [' german two-seater aircraft'], [' classic hits 106.3'], [' 31'], [' 19'], [' chevrolet'], [' england'], [' 925'], [' "the harvest"'], [' china'], [' chicago stags'], [' 3'], [' 240'], [' hanrapetakan stadium, yerevan, armenia'], [' 16'], [' dempsey'], [' 4'], [' 2'], [' 2'], [' massachusetts turnpike'], [' 3'], [' 6'], [' nieuport serial number 3958'], [' allianz riviera'], [' robin schembera'], [' 21'], [' 100 m'], [' 19-9']

Acierto del 49.4%, se nota que el fine-tuning del modelo afecta mucho a los resultados que obtengamos.

In [ ]:
print(all_predictions)
print(true_labels)

all_predictions, true_labels = transformar_respuestas(all_predictions, true_labels)

print('Resultados transformados:')
print(all_predictions)
print(true_labels)

mlb = MultiLabelBinarizer()
true_labels_binary = mlb.fit_transform(true_labels)
all_predictions_binary = mlb.transform(all_predictions)

accuracy = accuracy_score(true_labels_binary, all_predictions_binary)
print(f"Accuracy: {accuracy}")

[[' 3'], [' 7'], [' 2001'], [' mr b. owen- jones'], [' new zealand'], [' 2013'], [' 100,000'], [' 4'], [' alco'], [' škoda octavia'], [' d'], [' afar'], [' 11'], [' feb 1 2013'], [' 209,945'], [' alamance regional medical center'], [' seattle international film festival'], [' lokomotiv moscow'], [' frank w. smith'], [' 2'], [' amager'], [' 4'], [' 2'], [' surround stakes, warwick stakes'], [' henry j. kaiser'], [' 1p/halley'], [' paul tracy'], [' brabham-repco'], [' 1 year'], [' 88'], [' você decide'], [' ragamuffin'], [' bob lemon'], [' 1'], [' niall english'], [' german two-seater aircraft'], [' classic hits 106.3'], [' 31'], [' 19'], [' chevrolet'], [' england'], [' 925'], [' "the harvest"'], [' china'], [' chicago stags'], [' 3'], [' 240'], [' hanrapetakan stadium, yerevan, armenia'], [' 16'], [' dempsey'], [' 4'], [' 2'], [' 2'], [' massachusetts turnpike'], [' 3'], [' 6'], [' nieuport serial number 3958'], [' allianz riviera'], [' robin schembera'], [' 21'], [' 100 m'], [' 19-9']

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['"every day i learn a little more about love"', '"the harvest"', '"the weekend aunt helen came"', '"under the sea"', '0.04', '1 year', '168', '182.4 km/h (113mph)', '19-9', '1926', '1934/35', '1958', '1:00:28 #', '1:48.79', '1p/halley', '2,770,000', '20 years', '2004', '2004, 2005, 2006, 2008, 2010, 2011, 2012, 2013', '2005', '21', '240', '258', '26 december 1960', '2849', '294.65', '31', '38', '4,110,015', '4,142', '40', '5 years', '5-7, 2-6', '5:37.86', '6.487 million', '636', '7cae', '7cae, 7dy, 7ex, 7ho, 7ht, 7la, 7nt, 7qn, 7qt, 7uv, 7zl', '81,338, 48,712, 20,000', '859,400', '88', '925', '978,000', 'alamance regional medical center', 'almasm rabilavich sharipov', 'amager', 'ann fargher (nzl), tine tollan (nor), antonette wilken (zim), guadalupe canseco (mex), claire izacard (fra), valerie mcfarland-beddoe (aus), alison childs (gbr), kerstin finke (frg), nicole kreil (aut), 

Podemos probar el mismo proceso pero con la versión vanilla de TAPEX-Large en vez del Base, para ver si obtenemos una mejora en los resultados.

In [12]:
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large")

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [13]:
!export EXP_NAME=model_large_finetuned

!python /content/drive/MyDrive/Proyecto_PLN/code/run_wikitablequestions_with_tapex.py \
  --do_train \
  --output_dir /content/model_large/$EXP_NAME \
  --model_name_or_path microsoft/tapex-large \
  --overwrite_output_dir \
  --per_device_train_batch_size 2 \
  --gradient_accumulation_steps 12 \
  --per_device_eval_batch_size 4 \
  --learning_rate 3e-5 \
  --logging_steps 10 \
  --save_steps 200 \
  --warmup_steps 200 \
  --predict_with_generate \
  --num_beams 5 \
  --weight_decay 1e-2 \
  --label_smoothing_factor 0.1 \
  --max_steps 1000 \
  --fp16

2023-12-25 10:27:47.309139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 10:27:47.309202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 10:27:47.310563: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-25 10:27:48.597853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/25/2023 10:27:51 - WARNING - __main__ -   Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
12/25/2023 10:27:51 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gp

In [11]:
model_path = "/content/model_large/"

tokenizer = TapexTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

A pesar de haber conseguido fine tunear el modelo, no lo he podido ejecutar sobre el conjunto de test porque sobrepasa la memoria de RAM disponible.

In [16]:
batch_size = 12

sample_size = 500
sample_indices = random.sample(range(len(test_data)), sample_size)
sampler = SubsetRandomSampler(sample_indices)

dataloader = DataLoader(test_data, batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)

all_predictions = []
true_labels = []

for batch in tqdm(dataloader, desc='Procesando'):
  queries, tables, labels = batch
  batch_predictions = question_answering(queries, tables)

  all_predictions.extend(batch_predictions)
  true_labels.extend(labels)

all_predictions = [[item] for item in all_predictions]
print(all_predictions)
print(true_labels)

Procesando: 100%|██████████| 42/42 [2:55:47<00:00, 251.13s/it]

[[' 5'], [' d. lawrence gunnels'], [' 31'], [' lata mangeshkar'], [' 10'], [' 1'], [' 15'], [' cfb kingston'], [' 8'], [' vickers armstrong'], [' steny hoyer'], [' 6'], [' united states'], [' 2'], [' september 23, 2001'], [' world indoor championships'], [' 1'], [' 6'], [' richard benedict'], [' united states'], [' 8'], [' engr.sikandar zaman'], [' 2001'], [' 2003'], [' mr issy kramer'], [' ontario fury'], [' drop bears'], [' greg'], [' north cairns tigers'], [' mg william a. mann'], [' 2'], [' japan'], [' 5'], [' alameda'], [' 2001'], [' 3'], [' with eliel saarinen'], [' 15'], [' 72-72-70-71'], [' agape christian academy'], [' new york giants'], [' 2'], [' no'], [' 5'], [' kyrylo fesenko'], [' 7'], [' 6'], [' 2'], [' 5'], [' totaal'], [' 3ª'], [' jordan'], [' uk'], [' 15 december 1985'], [' jo bonnier'], [' 7'], [' 2'], [' 4'], [' fiat 500 1.4 lounge 3d, fiat 500 1.4 sport,  citroen c4 2.0 sx 5dr 6sp a'], [' 1st'], [' warriors'], [' 3'], [' 1'], [' 17'], [' 5'], [' diesel'], [' eric v

In [17]:
print(all_predictions)
print(true_labels)

all_predictions, true_labels = transformar_respuestas(all_predictions, true_labels)

print('Resultados transformados:')
print(all_predictions)
print(true_labels)

mlb = MultiLabelBinarizer()
true_labels_binary = mlb.fit_transform(true_labels)
all_predictions_binary = mlb.transform(all_predictions)

accuracy = accuracy_score(true_labels_binary, all_predictions_binary)
print(f"Accuracy: {accuracy}")

[[' 5'], [' d. lawrence gunnels'], [' 31'], [' lata mangeshkar'], [' 10'], [' 1'], [' 15'], [' cfb kingston'], [' 8'], [' vickers armstrong'], [' steny hoyer'], [' 6'], [' united states'], [' 2'], [' september 23, 2001'], [' world indoor championships'], [' 1'], [' 6'], [' richard benedict'], [' united states'], [' 8'], [' engr.sikandar zaman'], [' 2001'], [' 2003'], [' mr issy kramer'], [' ontario fury'], [' drop bears'], [' greg'], [' north cairns tigers'], [' mg william a. mann'], [' 2'], [' japan'], [' 5'], [' alameda'], [' 2001'], [' 3'], [' with eliel saarinen'], [' 15'], [' 72-72-70-71'], [' agape christian academy'], [' new york giants'], [' 2'], [' no'], [' 5'], [' kyrylo fesenko'], [' 7'], [' 6'], [' 2'], [' 5'], [' totaal'], [' 3ª'], [' jordan'], [' uk'], [' 15 december 1985'], [' jo bonnier'], [' 7'], [' 2'], [' 4'], [' fiat 500 1.4 lounge 3d, fiat 500 1.4 sport,  citroen c4 2.0 sx 5dr 6sp a'], [' 1st'], [' warriors'], [' 3'], [' 1'], [' 17'], [' 5'], [' diesel'], [' eric v

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['"hog"', '"intro"', '"the harvest"', '"young forever"', '0.5', '14 days', '15:26.62 q, 15:27.25 q, 15:27.30 q, 15:27.75 q, 15:27.75 q, 15:29.03 q, 15:29.28 q, 15:29.37 q, 15:29.62 q, 15:29.62 q, 15:30.56 q, 15:32.00 q, 15:32.01 q, 15:32.13 q, 15:32.25 q, 15:32.71 q', '17 july 2010', '1936-1962', '1938/39', '1988-1989 season', '1st', '2.05', '20 august 1960', '2008-09', '21', '22', '22 years', '23rd', '25 728', '27 october 1918', '274.3', '3 years', '31', '318', '32', '350,000', '3776', '39', '39.092', '40', '46 lost', '48', '48.6%', '5 days', '5 years', '50', '58', '6.487 million', '62', '72-72-70-71', '725tds, 730d, 740d', '75,053, 75,053,75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053, 75,053', '7la', '8 619, 108 412, 158 671, 150 059, 158 671, 152 292, 130 580, 147 5